In [7]:
import numpy as np
import scipy.stats as sps

class InputNeuron(object):
    
    def __init__(self, nnet, spike_train):
        self.spike_train = spike_train
        self.output_spikes_times = []
        self.net = nnet
        
    def set_spike_train(self, spike_train):
        self.spike_train = spike_train
        self.output_spikes_times = []
        
    def step(self):
        if self.spike_train[self.net.global_time] == 1:
            self.output_spikes_times.append(self.net.global_time)
    
    def get_spikes(self):
        return self.output_spikes_times

class Neuron(object):
    
    def __init__(self, nnet):
        self.input_spikes = [] # pairs time, intensity
        self.output_spikes_times = []
        self.net = nnet
        self.potential = 0
        self.threshold = 1
        self.tau_m = 4
        self.tau_s = 2
        self.tau_r = 20
        self.time_scale = 1./9 # time unit is 100 ms
        self.history = [0]
    
    #очистить все после предыдущего запуска сети
    def restart(self):
        self.input_spikes = [] # pairs time, intensity
        self.output_spikes_times = []
        self.history = [0]
    
    def receive_spike(self, intensity):
        self.input_spikes.append((self.net.global_time, intensity))
    
    def step(self):
        self.potential = 0
        self.spike = False
        
        
        for spike_time, intensity in self.input_spikes:
            self.potential += self.eps(self.net.global_time - spike_time) * intensity
            
            
        for spike_time in self.output_spikes_times:
            self.potential += self.nu(self.net.global_time - spike_time)
        
        if self.potential > self.threshold:
            self.output_spikes_times.append(self.net.global_time)
            self.spike = True
        
        
        self.history.append(self.potential)
    
    def eps(self, time):
        s = time * self.time_scale # - delay
        return (np.exp(-np.abs(s)/self.tau_m) - np.exp(-np.abs(s)/self.tau_s)) * (s > 0)
    
    def grad_eps(self, time):
        s = time * self.time_scale
        
        return (np.exp(-np.abs(s)/self.tau_m) * (-1/self.tau_m) - np.exp(-np.abs(s)/self.tau_s) * (-1/self.tau_s)) * (s > 0)
    
    def nu(self, time):
        s = time * self.time_scale
        return -self.threshold * np.exp(-s/self.tau_r) * (s > 0)
    
    def get_spikes(self):
        return self.output_spikes_times


class Connection(object):

    def __init__(self, nnet, input_neuron, output_neuron,
                 weights=[1], delays=[1]): # weights and delays are scaled
        self.weights = weights
        self.delays = delays
        self.input_neuron = input_neuron
        self.output_neuron = output_neuron
        self.net = nnet
    
    def step(self):
        spikes = self.input_neuron.get_spikes()
        for spike_time in spikes:
            for weight, delay in zip(self.weights, self.delays):
                if spike_time + delay == self.net.global_time:
                    self.output_neuron.receive_spike(weight)
    
    def gradient(self, expected_spike_time):
        output_spikes = np.array(self.output_neuron.get_spikes())
        input_spikes = np.array(self.input_neuron.get_spikes())
        assert len(output_spikes) > 0
        output_spike_time = output_spikes[0]
        eps = self.output_neuron.eps
        grad_eps = self.output_neuron.grad_eps
        grad_numerator = 0
        grad_denumerator = 0
        for i in range(len(input_spikes)):
            grad_numerator += eps(output_spike_time - input_spikes[i] - self.delays)
            grad_denumerator += self.weights * grad_eps(output_spike_time - input_spikes[i] - self.delays) # sum over all delays and input axons for output_neuron
        grad = -grad_numerator / np.maximum(0.1, grad_denumerator) * (output_spike_time - expected_spike_time) # leaning rate will be in a learning algorithm
        return grad
    
    ############################ Learning algoriphm
    # ссылка на статью http://machinelearning.wustl.edu/mlpapers/paper_files/IM11.pdf стр.5
    # https://jivp-eurasipjournals.springeropen.com/articles/10.1186/s13640-015-0059-4
    def STDP():
        output_spikes = np.array(self.output_neuron.get_spikes())
        input_spikes = np.array(self.input_neuron.get_spikes())
        weights = np.array(self.weights)
        w_max = weights.max()
        w_min = weights.min()
        out_spikes_num = len(output_spikes)
        assert out_spikes_num > 0
        
        tau_post_before = 0
        tau_post_after = output_spikes[0]
        out_ind = 0
        
        # для каждого синапса нужно поменять его вес
        for j in range(len(weights)):
            for i, tau_pre in enumerate(input_spikes):
                ##################### тут нужно посмотреть, что делать со входящими спайками, которые пришли до первого 
                ##################### исходящего, и со спайками, которые пришли после после последнего исходящего
                if (tau_pre > tau_post_after):
                    tau_post_before = tau_post_after
                    if (out_ind < out_spikes_num - 1):
                        out_ind += 1
                        tau_post_after = output_spikes[out_ind]
                    else:
                        tau_post_after = 0
                delta_w = -1.0*etta*(self.tau_min-(tau_pre-tau_post_after))(0 <= tau_pre-tau_post_after <=tau_min)\
                        +etta*(self.tau_plus+(tau_pre-tau_post_after))(-1.*tau_plus <= tau_pre-tau_post_after <= 0)\
                         -1.0*etta*(self.tau_min-(tau_pre-tau_post_before))(0 <= tau_pre-tau_post_before <=tau_min)\
                        +etta*(self.tau_plus+(tau_pre-tau_post_before))(-1.*tau_plus <= tau_pre-tau_post_before <= 0)

                w_old = self.weights[j]
                self.weights[j] = w_old + sigma*delta_w*((w_max - w_old)*(delta_w > 0) + (w_old - w_min)*(delta_w <= 0))


In [8]:
from functools import reduce
class InputLayer(object):
    def __init__(self, nnet, shape):
        self.net = nnet
        self.shape = shape 
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.ndarray(shape=self.shape, dtype=InputNeuron, buffer=np.array([InputNeuron(self.net, []) for i in np.arange(self.neur_size)]))
            
    def make_spike_train(self, freq, t, t_max):
        if t==0:
            return np.zeros(t_max)
        return sps.bernoulli.rvs(freq*t, size=t_max)
    
    def new_input(self, arg, t=100, t_max=1000):
        for i, f in enumerate(arg):
            for j, l in enumerate(f):
                for k, m in enumerate(l):
                    self.neurons[i][j][k].set_spike_train(make_spike_train(arg[i][j][k], t, t_max))
    
    def step(self):
        for neur in self.neurons.reshape((self.neur_size)):
            neur.step()
            
    def learning():
        pass

In [9]:
class Conv2DLayer(object):
    # Формат весов: w[nk][h][i][j], где nk - фильтр, h - номер фильтра на предыдущем слое, i, j - координаты весов в фильтре
    def __init__(self, nnet, input_layer, num_filters, filter_shape, weights):
        self.net = nnet
        
        self.weights = weights
        if(len(self.weights.shape) < 5):
            self.weights = self.weights.reshape(np.append(weights.shape, 1))
        
        self.shape = (num_filters, input_layer.shape[1]-filter_shape[0]+1, input_layer.shape[2]-filter_shape[1]+1)
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)]).reshape(self.shape)
        
        self.connections = []
        
        for nk, kernel in enumerate(self.neurons):
            for i, row in enumerate(kernel):
                for j, neuron in enumerate(row):   # соединяем с предыдущим слоем
                    self.connections += [Connection(self.net, input_layer.neurons[l][i+p][j+q],neuron, self.weights[nk][l][p][q])\
                                         for l in np.arange(input_layer.shape[0]) for p in np.arange(filter_shape[0])\
                                         for q in np.arange(filter_shape[1])] 
                    
    def restart(self):
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.restart()
    
    def step(self):
        for conn in self.connections:
            conn.step()
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.step()
            
    def learning():
        for conn in self.connections:
            conn.STDP()

In [10]:
class SubSampling2DLayer(object):
    def __init__(self, nnet, input_layer, pool_size):
        self.net = nnet
        self.shape = input_layer.shape // np.append([1], pool_size)
        self.neur_size = reduce(lambda res, x: res*x, self.shape, 1)
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)]).reshape(self.shape)
        
        self.conn_weight = 1 / (pool_size[0] * pool_size[1])
        
        self.connections = []
        
        for nk, kernel in enumerate(self.neurons):
            for i, row in enumerate(kernel):
                for j, neuron in enumerate(row):   
                    self.connections += [Connection(self.net,input_layer.neurons[l][i*pool_size[0]+p][j*pool_size[1]+q],neuron,\
                                                    [self.conn_weight])\
                                         for l in np.arange(input_layer.shape[0]) for p in np.arange(pool_size[0])\
                                         for q in np.arange(pool_size[1])] 
                    
    def restart(self):
        for i, neur in enumerate(self.neurons.reshape((self.neur_size))):
            neur.restart()
        
    def step(self):
        for conn in self.connections:
            conn.step()
        for neur in self.neurons.reshape((self.neur_size)):
            neur.step()
            
    def learning():
        for conn in self.connections:
            conn.STDP()

In [11]:
class DenseLayer(object):
    #Формат весов: w[i][j],  где i - номер нейрона на предыдущем слое, j - номер нейрона на текущем слое
    def __init__(self,nnet, input_layer, num_units, weights):
        self.net = nnet
        self.shape = [num_units]
        self.neur_size = num_units
        self.neurons = np.array([Neuron(self.net) for i in np.arange(self.neur_size)])
        
        if(len(weights.shape) < 3):
            weights = weights.reshape(np.append(weights.shape, 1))
        
        self.connections = [Connection(self.net, input_neuron, output_neuron, weights[i][j])\
                            for i, input_neuron in enumerate(input_layer.neurons.reshape((input_layer.neur_size)))\
                            for j, output_neuron in enumerate(self.neurons)]
        
    def restart(self):
        for neur in self.neurons:
            neur.restart()
        
    def step(self):
        for conn in self.connections:
            conn.step()
        for neur in self.neurons:
            neur.step()
            
    def learning():
        for conn in self.connections:
            conn.STDP()
    

In [12]:
class NNet(object):
    def __init__(self, shape):
        self.layers = [InputLayer(self, shape)]
        self.global_time = 0
    
    def add_convolution(self, weights):
        num_filters = weights.shape[0]
        filter_shape = weights.shape[2:4]
        self.layers.append(Conv2DLayer(self, self.layers[-1], num_filters, filter_shape, weights))
        
    def add_subsampling(self, pool_size):
        self.layers.append(SubSampling2DLayer(self, self.layers[-1], pool_size))
        
    def add_dense(self, weights):
        num_units = weights.shape[1]
        self.layers.append(DenseLayer(self, self.layers[-1], num_units, weights))
    
    def get_output_for(self, data, t_max):
        self.global_time = 0
        self.layers[0].new_input(data)
        for l in self.layers[1:]:
            l.restart()
        for t in np.arange(t_max):
            for layer in self.layers:
                layer.step()
            self.global_time += 1
        result = [neur.get_spikes() for neur in self.layers[-1].neurons.reshape((self.layers[-1].neur_size))]
        return result
    
    def classify(self, data, t_max):
        self.global_time = 0
        self.layers[0].new_input(data)
        for l in self.layers[1:]:
            l.restart()
        ans = np.zeros(self.layers[-1].neur_size)
        for t in np.arange(t_max):
            for layer in self.layers:
                layer.step()
            for i, neur in enumerate(self.layers[-1].neurons):
                if len(neur.getspikes()) > 0:
                    ans.append(i)
            if(len(ans) > 0):
                return ans[0], t
            self.global_time += 1
            
    def learning():
        for layer in self.layers:
            layer.learning()

In [ ]:
# learning test on XOR function
inputs = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
targets = np.array([[0.], [1.], [1.], [0.]])
nn = NNet((1, 4, 4))

In [7]:
import lasagne

def spiking_from_lasagne(input_net):
    input_layers = lasagne.layers.get_all_layers(input_net)
    weights = lasagne.layers.get_all_param_values(input_net)
    spiking_net = NNet(input_layers[0].shape[-3:])
    convert_layers = {lasagne.layers.conv.Conv2DLayer : spiking_net.add_convolution,\
                      lasagne.layers.dense.DenseLayer : spiking_net.add_dense}
    
    #номер элемента в общем массиве весов, в котором хранятся веса текущего слоя
    i = 0
    
    for l in input_layers[1:]:
        if(type(l) == lasagne.layers.pool.Pool2DLayer or type(l) == lasagne.layers.pool.MaxPool2DLayer):
            spiking_net.add_subsampling(l.pool_size)
        else:
            convert_layers[type(l)](weights[i])
            i+=2
            
    return spiking_net

In [ ]:
nn = NNet((1, 4, 4))
nn.add_convolution(2, (3, 3), [[[1, 1, 1], [1, 0, 1], [0, 0, 1]], [[1, 1, 1], [1, 1, 1], [1, 1, 1]]])
nn.add_subsampling((2, 2))
nn.add_dense(1, [[100], [100]])

In [ ]:
train = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
inp = np.array([train]*16).reshape(1, 4, 4, 18)

In [11]:
import theano.tensor as T
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)
conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters=2, filter_size=3, name='conv_1', 
                                    nonlinearity=lasagne.nonlinearities.rectify)
max_pool_1 = lasagne.layers.MaxPool2DLayer(conv_1, pool_size=(2,2), name='max_pool_1')
conv_2 = lasagne.layers.Conv2DLayer(max_pool_1, num_filters=3, filter_size=3, nonlinearity=lasagne.nonlinearities.rectify, 
                                    name='conv_2')
max_pool_2 = lasagne.layers.MaxPool2DLayer(conv_2, pool_size=(2,2), name='max_pool_2')
dense_output = lasagne.layers.DenseLayer(max_pool_2, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

#print(max_pool_1.input_shape)
l = lasagne.layers.get_all_layers(max_pool_2)
max_pool_1.pool_size

(2, 2)

In [12]:
nnnet = spiking_from_lasagne(dense_output)

In [13]:
for i in nnnet.layers:
    print(i, i.shape)

(<__main__.InputLayer object at 0x107c2b650>, (1, 28, 28))
(<__main__.Conv2DLayer object at 0x107cf0a90>, (2, 26, 26))
(<__main__.SubSampling2DLayer object at 0x107cf0ad0>, array([ 2, 13, 13]))
(<__main__.Conv2DLayer object at 0x107db1310>, (3, 11, 11))
(<__main__.SubSampling2DLayer object at 0x1082c8950>, array([3, 5, 5]))
(<__main__.DenseLayer object at 0x108419490>, [10])


In [15]:
[3, 4, 6] * 0

[]

In [221]:
import scipy.stats as sps
sps.bernoulli.rvs(90/255, size=255)

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0])

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [46]:
np.dot([1, 2], [2, 3])

8

In [50]:
X_train = mnist.train.images.reshape(55000, 1, 28, 28)
y_train = [np.dot(j, np.arange(10)) for j in mnist.train.labels]
X_test = mnist.test.images.reshape(10000, 1, 28, 28)
y_test = [np.dot(j, np.arange(10)) for j in mnist.test.labels]

In [51]:
import theano
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters=8, filter_size=3, name='conv_1', 
                                    nonlinearity=lasagne.nonlinearities.rectify)

max_pool_1 = lasagne.layers.Pool2DLayer(conv_1, pool_size=(2,2), name='max_pool_1', mode='average_inc_pad')

conv_2 = lasagne.layers.Conv2DLayer(conv_1, num_filters=8, filter_size=3, nonlinearity=lasagne.nonlinearities.rectify, 
                                    name='conv_2')
max_pool_2 = lasagne.layers.Pool2DLayer(conv_2, pool_size=(2,2), name='max_pool_2', mode='average_inc_pad')
dense_output = lasagne.layers.DenseLayer(max_pool_2, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

In [52]:
y_predicted = lasagne.layers.get_output(dense_output)
all_weights = lasagne.layers.get_all_params(dense_output)
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
updates_sgd = lasagne.updates.rmsprop(loss, all_weights,learning_rate=0.01)
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)
accuracy_fun = theano.function([input_X,target_y],accuracy)
from random import shuffle
import math
def iterate_minibatches(X, y, batchsize):
    
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for start_idx in range(0, len(X) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield X[excerpt], y[excerpt]

import time
from tqdm import tqdm

num_epochs = 5 #количество проходов по данным

batch_size = 30 #размер мини-батча

for epoch in tqdm(range(num_epochs)):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size):
        
        
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

  0%|          | 0/5 [00:00<?, ?it/s]


TypeError: only integer scalar arrays can be converted to a scalar index